In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# Identifying Keywords from News Articles

Idea:
- Webscrape text from article links
- Use TFIDF/NER to idenitfy keywords from the text for tagging purposes
- Use Keywords to create a summary


## Import Data

In [3]:
date_range = pd.date_range('01-01-2006', datetime.today().date())
df = pd.read_csv("../actions.csv")
df["date"] = pd.to_datetime(df["date"])
df.head()

,Unnamed: 0,id,date,sources,actions,struggles,employment_types,description,online,locations,companies,workers,tags,author,latlngs,addresses
0,0,1,1979-09-01,['https://twitter.com/clancynewyork/status/117...,['protest'],['ethics'],['white_collar_workers'],"IBM workers formed an organization, IBM: Speak...",None,['worldwide'],['ibm'],None,['international_solidarity'],['nataliyaned'],None,None
1,1,2,1986-09-16,['https://www.nytimes.com/1986/09/16/science/s...,['open_letter'],['ethics'],['white_collar_workers'],Fourteen employees at AT&T sign a letter warni...,None,['usa'],['at&t'],14,None,['nataliyaned'],"[(45.5733162, -122.5587111463347)]","['10025, Northeast Cascades Parkway, Portland,..."
2,2,3,1986-09-16,['https://www.nytimes.com/1986/09/16/science/s...,['open_letter'],['ethics'],['white_collar_workers'],"A group of computer professionals, led by Comp...",None,['usa'],None,30,None,['nataliyaned'],"[(39.7837304, -100.4458825)]",['United States']
3,3,4,1994-01-24,['https://www.nytimes.com/1994/06/12/magazine/...,['open_letter'],['ethics'],['white_collar_workers'],Computer Professionals for Social Responsibili...,True,None,None,50000,None,['nataliyaned'],None,None
4,4,5,1998-11-23,['http://www.cnn.com/tech/computing/9811/23/ms...,['legal_action'],"['unfair_labor_practices', 'pay_and_benefits']",['contract_workers'],A law firm representing 10 current and former ...,None,['usa'],['microsoft'],10,None,['organizejs'],"[(35.139622349999996, -80.92306993327955)]","['Microsoft, Yorkwood, Charlotte, Mecklenburg ..."


## Links to Explore

In [4]:
df["sources"]

0      ['https://twitter.com/clancynewyork/status/117...
1      ['https://www.nytimes.com/1986/09/16/science/s...
2      ['https://www.nytimes.com/1986/09/16/science/s...
3      ['https://www.nytimes.com/1994/06/12/magazine/...
4      ['http://www.cnn.com/tech/computing/9811/23/ms...
                             ...                        
277    ['https://rabble.ca/news/2020/06/foodsters-uni...
278    ['https://medium.com/@macprac/open-letter-the-...
279    ['https://medium.com/@catalina.brennan.gatica/...
280    ['https://www.theverge.com/2020/6/22/21299736/...
281    ['https://medium.com/@CoalitionForCriticalTech...
Name: sources, Length: 282, dtype: object

In [5]:
df["sources"][0]

"['https://twitter.com/clancynewyork/status/1175872040814993408?s=19, http://www-cs-students.stanford.edu/~cale/cs201/apartheid.comp.html']"

## Webscrape for source text

In [6]:
url = "http://www-cs-students.stanford.edu/~cale/cs201/apartheid.comp.html"
response = requests.get(url)

In [7]:
soup = BeautifulSoup(response.text, "html.parser")

In [8]:
print(soup)

<html>
<title>The Use of Computers to Support Oppression</title>
<body alink="#FFFF00" background="pictures/olive_paper.gif" link="#F2F295" text="#FFFFFF" vlink="#C9B720">
<center>
<h1><b>The Use of Computers to Support Oppression</b></h1><p>
</p></center>

Computer technology enabled the government to organize and enforce such an atrocious system of segregation and control.

<blockquote>
More than any other single technological advancement, the computer fostered the concentration of administrative power in the hands of Africa's white elite. <a href="apartheid.bib.html"><b>[NAR82]</b></a>
</blockquote>

Despite the U.N. arms embargoes, American computers were in widespread use throughout South Africa.  The United States was the largest supplier of computers used in South Africa. American computers were in use in virtually every governmental agency, the police system, and the military, all of which contributed to the control system known as apartheid.  Computer technology did not merely

## Create Sources & Text DataFrame

In [9]:
df["sources"] = df["sources"] = df["sources"].astype(str).str.strip("[]\'")

In [10]:
sources = []
for i in df["sources"].unique():
    sources.extend(i.split(", "))

sources = list(set(sources))

In [11]:
sources_df = pd.DataFrame({"Source": sources})
sources_df.head()

,Source
0,https://www.nbcnews.com/news/all/amazon-worker...
1,https://www.vox.com/recode/2019/6/19/18691870/...
2,https://www.cbsnews.com/news/what-happened-aft...
3,https://nowtoronto.com/food-and-drink/food/foo...
4,https://medium.com/@socialcontractcr/daemos-cr...


In [14]:
from tqdm import tqdm

texts = []
for url in tqdm(sources_df["Source"].to_numpy()):
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")
    texts.append(soup)
    
print(len(sources), len(texts))

 36%|█████████████████████████████▏                                                  | 175/480 [02:55<15:10,  2.99s/it]

ConnectionError: HTTPConnectionPool(host='wiki.wearedynamo.org', port=80): Max retries exceeded with url: /index.php/guidelines_for_academic_requesters (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002D537180D48>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [16]:
# Problem URL
sources[175]

# Figure out how to define a time-out/connection fail situation (Text will be "Null")

'http://wiki.wearedynamo.org/index.php/guidelines_for_academic_requesters'

In [40]:
sources_df["Text"] = texts

ConnectionError: ('Connection aborted.', OSError("(10060, 'WSAETIMEDOUT')"))

In [ ]:
sources_df.head()